Test model

In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import torch
from model import Model
path = '../data/train_data.pkl'
path2 = '../data/val_data.pkl'
noisy_imgs_1, noisy_imgs_2 = torch.load(path)
noisy_imgs_1, noisy_imgs_2 = noisy_imgs_1.float(), noisy_imgs_2.float()
test, truth = torch.load(path)
test, truth = test.float(), truth.float()

In [17]:
def sample(tensor1, tensor2, k):
    perm = torch.randperm(tensor1.size(0))
    idx = perm[:k]
    return tensor1[idx], tensor2[idx]

In [20]:
m = Model()
s1, s2 = sample(noisy_imgs_1, noisy_imgs_2, 500)
m.train(s1, s2, 50)

0
10
20
30
40


In [25]:
from others.pnsr import psnr
psnr(m.predict(test), truth)